In [2]:
!pip -q install xgboost


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install -U sagemaker


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"AbaloneModelPackageGroupName"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
!mkdir -p data1

In [6]:


%%time

from time import gmtime, strftime
import re
from datetime import datetime, timedelta, timezone

local_path = "/root/aai540/finalproject/Churn_Modelling.xls"

s3 = boto3.resource("s3")

base_uri = f"s3://{default_bucket}/churnmodeling-{datetime.utcnow():%Y-%m-%d-%H%M}"
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-us-east-1-994366592132/churnmodeling-2024-02-18-0511/Churn_Modelling.xls
CPU times: user 201 ms, sys: 27.3 ms, total: 228 ms
Wall time: 450 ms


In [7]:
import csv

# Define the input and output file paths
input_file = "data/batch_data.csv"
output_file = "data/batch_data1.csv"

# Read the input CSV file, remove the first column, and write to the output CSV file
with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    for row in reader:
        # Exclude the first column from each row
        new_row = row[1:]
        writer.writerow(new_row)

print("First column removed and saved to batch_data1.csv")


First column removed and saved to batch_data1.csv


In [8]:
local_path = "data/batch_data1.csv"

s3 = boto3.resource("s3")

batch_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(batch_data_uri)

s3://sagemaker-us-east-1-994366592132/churnmodeling-2024-02-18-0511/batch_data1.csv


In [9]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)
batch_data = ParameterString(
    name="BatchData",
    default_value=batch_data_uri,
)
mse_threshold = ParameterFloat(name="MseThreshold", default_value=6.0)

# Processing step for Feature Engineering

In [10]:
!mkdir -p code

In [11]:
%%writefile code/preprocessing.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

if __name__ == "__main__":
    base_dir = "/opt/ml/processing"

    # Load the original data
    df = pd.read_csv(f"{base_dir}/input/Churn_Modelling.xls")

    # Rename columns for clarity
    df.columns = ['RowNumber', 'CustomerID', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure',
                  'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']
    
    # Add print statements to inspect data consistency
    print("Original data shape:", df.shape)

       
    df.drop(columns=["RowNumber", "Surname"], inplace=True)
    df = pd.get_dummies(df)
    # Check the dimensionality after one-hot encoding
    print("Dimensionality after one-hot encoding:", X.shape)
    df.drop(columns="Gender_Male", inplace=True)
    
    # Print the first 5 rows of each CSV file
    print("First 5 rows of train.csv:")
    print(df.head())
    
    # Check for missing values or NaNs
    print("Missing values:\n", df.isnull().sum())
    
    # Ensure data types are as expected
    print("Data types:\n", df.dtypes)
    
    # Inspect unique values for categorical variables
    print("Unique values for 'Exited' column:", df['Exited'].unique())

    # Split features and target
    X = df.drop(columns="Exited")
    y = df["Exited"]

    # Split data into train, test, and validation sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

    # Drop CustomerID column
    X_train.drop(columns="CustomerID", inplace=True)
    X_val.drop(columns="CustomerID", inplace=True)
    # Drop CustomerID column from the test set
    X_test.drop(columns="CustomerID", inplace=True)
    
    # Print the first 5 rows of each CSV file
    print("First 5 rows of train.csv:")
    print(pd.concat([y_train, X_train], axis=1).head())

    print("First 5 rows of validation.csv:")
    print(pd.concat([y_val, X_val], axis=1).head())

    print("First 5 rows of test.csv:")
    print(pd.concat([y_test, X_test], axis=1).head())

    # Save data to CSV files
    train_file = f"{base_dir}/train/train.csv"
    pd.concat([y_train, X_train], axis=1).to_csv(train_file, header=False, index=False)

    validation_file = f"{base_dir}/validation/validation.csv"
    pd.concat([y_val, X_val], axis=1).to_csv(validation_file, header=False, index=False)

    test_file = f"{base_dir}/test/test.csv"
    pd.concat([y_test, X_test], axis=1).to_csv(test_file, header=False, index=False)

    print("Preprocessing completed.")


Overwriting code/preprocessing.py


In [12]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-abalone-process",
    role=role,
    sagemaker_session=pipeline_session,
)

In [13]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="code/preprocessing.py",
)

step_process = ProcessingStep(name="BankChurnPredictionProcess", step_args=processor_args)

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


# Define a training step to train a model

In [14]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

model_path = f"s3://{default_bucket}/BankChurnPredictionTrain"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=pipeline_session,
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
)

train_args = xgb_train.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

In [15]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="BankChurnPredictionTrain",
    step_args=train_args,
)

# define a model evaluation step to evaluate the trained model

In [16]:
%%writefile code/evaluation.py
import json
import pathlib
import pickle
import tarfile

import joblib
import numpy as np
import pandas as pd
import xgboost

from sklearn.metrics import mean_squared_error


if __name__ == "__main__":
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")

    model = pickle.load(open("xgboost-model", "rb"))
    
    # Load the test data
    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)

    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)

    X_test = xgboost.DMatrix(df.values)
    
    # Perform predictions
    predictions = model.predict(X_test)
    
    print("Test Data Shape:", X_test.shape)
    print("Model Predictions:", predictions)

    
    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, predictions)
    std = np.std(y_test - predictions)
    
    # Save evaluation results
    report_dict = {
        "regression_metrics": {
            "mse": {"value": mse, "standard_deviation": std},
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting code/evaluation.py


In [17]:
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="script-bankchurn-eval",
    role=role,
    sagemaker_session=pipeline_session,
)

eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="code/evaluation.py",
)

In [ ]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)
step_eval = ProcessingStep(
    name="BankChurnPredictionEval",
    step_args=eval_args,
    property_files=[evaluation_report],
)

## Define a Create Model Step to Create a Model

In [ ]:
from sagemaker.model import Model

model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

In [ ]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep

step_create_model = ModelStep(
    name="BankChurnPredictionModel",
    step_args=model.create(instance_type="ml.m5.large", accelerator_type="ml.eia1.medium"),
)

## Define a Transform Step to Perform Batch Transformation

In [ ]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=f"s3://{default_bucket}/BankChurnTransform",
)

In [ ]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="BankChurnTransform", transformer=transformer, inputs=TransformInput(data=batch_data)
)

## Define a Register Model Step to Create a Model Package

In [ ]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)
step_register = ModelStep(name="BankChurnRegisterModel", step_args=register_args)

## Define a Fail Step to Terminate the Pipeline Execution and Mark it as Failed

In [ ]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join

step_fail = FailStep(
    name="BankChurnMSEFail",
    error_message=Join(on=" ", values=["Execution failed due to MSE >", mse_threshold]),
)

## Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry, Or Terminate the Execution in Failed State

In [ ]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=mse_threshold,
)

step_cond = ConditionStep(
    name="BankChurnMSECond",
    conditions=[cond_lte],
    if_steps=[step_register, step_create_model, step_transform],
    else_steps=[step_fail],
)

## Define a Pipeline of Parameters, Steps, and Conditions

In [ ]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"BankChurnPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        batch_data,
        mse_threshold,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
)

### (Optional) Examining the pipeline definition

In [ ]:
import json


definition = json.loads(pipeline.definition())
definition

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
execution.wait()

In [32]:
execution.list_steps()

[{'StepName': 'BankChurnTransform',
  'StartTime': datetime.datetime(2024, 2, 18, 4, 9, 44, 487000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 2, 18, 4, 15, 3, 531000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'AttemptCount': 1,
  'FailureReason': 'ClientError: ClientError: See job logs for more information',
  'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:us-east-1:994366592132:transform-job/pipelines-z4tjvsom8p0n-BankChurnTransform-4aLXP7OoCN'}}},
 {'StepName': 'BankChurnRegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2024, 2, 18, 4, 9, 42, 573000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 2, 18, 4, 9, 43, 785000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:994366592132:model-package/AbaloneModelPackageGroupName/3'}}},
 {'StepName': 'BankChurnPredictionModel-CreateModel',
  'StartTime': datetime.datetime(2024, 2, 18, 4, 9, 42, 573000, 

## Examine the Evaludation

In [33]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file(
    "{}/evaluation.json".format(
        step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    )
)
pprint(json.loads(evaluation_json))

{'regression_metrics': {'mse': {'standard_deviation': 0.2964536476698355,
                                'value': 0.08821944492017866}}}


## Lineage

In [34]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

{'StepName': 'BankChurnPredictionProcess', 'StartTime': datetime.datetime(2024, 2, 18, 3, 57, 25, 405000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 2, 18, 4, 1, 53, 825000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:994366592132:processing-job/pipelines-z4tjvsom8p0n-BankChurnPredictionP-X1VwvX6zdv'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...a4a622cd86e393a6935f228/preprocessing.py,Input,DataSet,ContributedTo,artifact
1,s3://...ling-2024-02-18-0357/Churn_Modelling.xls,Input,DataSet,ContributedTo,artifact
2,68331...com/sagemaker-scikit-learn:1.2-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...n/BankChurnPredictionProcess/output/test,Output,DataSet,Produced,artifact
4,s3://...ChurnPredictionProcess/output/validation,Output,DataSet,Produced,artifact
5,s3://.../BankChurnPredictionProcess/output/train,Output,DataSet,Produced,artifact


{'StepName': 'BankChurnPredictionTrain', 'StartTime': datetime.datetime(2024, 2, 18, 4, 1, 54, 613000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 2, 18, 4, 4, 42, 967000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:994366592132:training-job/pipelines-z4tjvsom8p0n-BankChurnPredictionT-cO5aDDLiOE'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...ChurnPredictionProcess/output/validation,Input,DataSet,ContributedTo,artifact
1,s3://.../BankChurnPredictionProcess/output/train,Input,DataSet,ContributedTo,artifact
2,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...edictionT-cO5aDDLiOE/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'BankChurnPredictionEval', 'StartTime': datetime.datetime(2024, 2, 18, 4, 4, 43, 516000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 2, 18, 4, 9, 39, 651000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:994366592132:processing-job/pipelines-z4tjvsom8p0n-BankChurnPredictionE-9sTRo9M0re'}}}


...

None

{'StepName': 'BankChurnRegisterModel-RegisterModel', 'StartTime': datetime.datetime(2024, 2, 18, 4, 9, 42, 573000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 2, 18, 4, 9, 43, 785000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 1, 'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:994366592132:model-package/AbaloneModelPackageGroupName/3'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...edictionT-cO5aDDLiOE/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
2,AbaloneModelPackageGroupName-3-PendingManualAp...,Input,Approval,ContributedTo,action
3,AbaloneModelPackageGroupName-1708056647-aws-mo...,Output,ModelGroup,AssociatedWith,context


{'StepName': 'BankChurnTransform', 'StartTime': datetime.datetime(2024, 2, 18, 4, 9, 44, 487000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 2, 18, 4, 15, 3, 531000, tzinfo=tzlocal()), 'StepStatus': 'Failed', 'AttemptCount': 1, 'FailureReason': 'ClientError: ClientError: See job logs for more information', 'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:us-east-1:994366592132:transform-job/pipelines-z4tjvsom8p0n-BankChurnTransform-4aLXP7OoCN'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...edictionT-cO5aDDLiOE/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
2,s3://...modeling-2024-02-18-0357/batch_data1.csv,Input,DataSet,ContributedTo,artifact
3,s3://...s-east-1-994366592132/BankChurnTransform,Output,DataSet,Produced,artifact


## Parameterized Executions

In [35]:
execution = pipeline.start(
    parameters=dict(
        ModelApprovalStatus="Approved",
    )
)

In [36]:
execution.wait()

WaiterError: Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression "PipelineExecutionStatus" we matched expected path: "Failed"

In [ ]:
execution.list_steps()

In [ ]:
execution = pipeline.start(parameters=dict(MseThreshold=3.0))

In [ ]:
try:
    execution.wait()
except Exception as error:
    print(error)

In [ ]:
execution.list_steps()